In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

In [2]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

In [3]:
random_state = 42

In [4]:
feature_set =  'feature_set_normalized_pca'
X_train = pd.read_csv( F'./data/{feature_set}/X_train_full.csv')
y_train = pd.read_csv( F'./data/{feature_set}/y_train.csv').values.ravel()

X_val = pd.read_csv( F'./data/{feature_set}/X_valid_full.csv')
y_val = pd.read_csv( F'./data/{feature_set}/y_valid.csv').values.ravel()

In [5]:
X_train.head()

,CARD_BRAND_Visa,CARD_COUNTRY_CODE_US,OUTLET_TYPE_Face to Face,BUSINESS_TYPE_Limited Liability Company (LLC),ACQUIRER_ID_ACQ2,CARDHOLDER_AUTH_METHOD_Online PIN,TRANSACTION_CURRENCY_GBP,ACQUIRER_ID_ACQ1,Time_of_day_00:00-05:59,Time_of_day_18:00-23:59,...,TRANSACTION_CURRENCY_JPY,PAYMENT_PERCENTAGE_MOTO,TRANSACTION_STATUS_Captured,MCC_CODE,CARD_COUNTRY_CODE_DK,CARD_COUNTRY_CODE_DK.1,DELIVERY_OVER_TWO_WEEKS_PERCENTAGE,TRANSACTION_STATUS_Authorized,CARD_COUNTRY_CODE_RO,ACQUIRER_ID_ACQ5
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.490877,0.0,0.0,0.243902,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.451564,0.0,0.0,0.439024,0.0,0.0,0.0
2,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.866667,0.0,0.542137,0.0,0.0,0.390244,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.0,0.522372,0.0,0.0,0.487805,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.000000,0.0,0.812120,0.0,0.0,0.463415,0.0,0.0,0.0


In [6]:
for n_esti in [4,8,16,32,64,128,256,512,1024]:
	forest_model = RandomForestRegressor(n_estimators=n_esti,random_state=random_state,verbose=1,n_jobs=-1)
	forest_model.fit(X_train,y_train)
 
	train_predictions = forest_model.predict(X_train)
	print(f'Train ROC score with {n_esti} estimators')
	print(roc_auc_score(y_train, train_predictions))
 
	val_predictions = forest_model.predict(X_val)
	print(f'Val ROC score with {n_esti} estimators')
	print(roc_auc_score(y_val, val_predictions))
	print("\n")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   16.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   16.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished


Train ROC score with 4 estimators
0.9921979113621809


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Val ROC score with 4 estimators
0.6373234648724021




[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   21.8s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   23.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.2s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


...

Train ROC score with 256 estimators
1.0


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    4.1s
[Parallel(n_jobs=8)]: Done 256 out of 256 | elapsed:    5.5s finished


Val ROC score with 256 estimators
0.6922401654847122




[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


KeyboardInterrupt: 